# ETL Project - MLS

In [268]:
# Import Dependencies
import pandas as pd

Data Cleaning Secetion

In [269]:
# Read CSV files in pandas
mmls_path = ('mens_major_league_soccer.csv')
mls_salaries_path = ('mls-salaries-2017.csv')
mmls_data = pd.read_csv(mmls_path, encoding="ISO-8859-1")
mls_salaries = pd.read_csv(mls_salaries_path, encoding="ISO-8859-1")

In [270]:
# Preview of the DataFrame
mmls_data_temp = mmls_data.iloc[4:]
mmls_data_temp2 = mmls_data_temp.rename(columns=mmls_data_temp.iloc[0]).drop(mmls_data_temp.index[0])
mmls_data_temp2.describe()

,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,PK,PKatt,CrdY,CrdR,Gls,Ast,G+A,G-PK,G+A-PK,Matches
count,588,588,588,588,588,588,588,588,588,588,...,588,588,588,588,588,588,588,588,588,588
unique,588,575,77,10,22,22,22,34,35,497,...,7,8,12,5,72,51,94,66,91,1
top,346,Ethan Finlay\Ethan-Finlay,us USA,DF,Vancouver,26,1990,1,0,90,...,0,0,0,0,0,0,0,0,0,Matches
freq,1,2,270,168,31,62,60,36,49,7,...,547,539,177,507,308,299,223,311,224,588


In [271]:
# Remove special characters from Player column and separate Player Names
mmls_data_temp2[['Name1','Name2']] = pd.DataFrame(mmls_data_temp2.Player.str.split(r'\\',1).tolist())
mmls_data_temp3 = mmls_data_temp2.drop(['Player', 'Name2'], axis=1)
mmls_data_df = mmls_data_temp3.rename(columns = {'Name1':'Player'})
mmls_data_df.head()

,Rk,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,...,PKatt,CrdY,CrdR,Gls,Ast,G+A,G-PK,G+A-PK,Matches,Player
5,1,us USA,DF,Sporting KC,25,1991,12,10,902,1,...,0,1,0,0.1,0.1,0.2,0.1,0.2,Matches,Kellyn Acosta
6,2,gh GHA,MF,Columbus,25,1991,20,14,1279,0,...,0,3,0,0,0,0,0,0,Matches,Luciano Acosta
7,3,gh GHA,DF,Columbus,22,1994,7,6,529,1,...,0,2,1,0.17,0,0.17,0.17,0.17,Matches,Tyler Adams
8,4,gh GHA,MFFW,Chicago,26,1990,30,24,2194,14,...,2,2,0,0.57,0.25,0.82,0.49,0.74,Matches,Seyi Adekoya
9,5,us USA,DF,Real Salt Lake,19,1997,17,16,1408,0,...,0,2,1,0,0.06,0.06,0,0.06,Matches,Fanendo Adi


In [272]:
mls_salaries.rename(columns = {'Unnamed: 6':'Player'}, inplace = True)
mls_salaries_df = mls_salaries.drop(['last_name','first_name'], axis=1)
mls_salaries_df.head()

,club,position,base_salary,guaranteed_compensation,Player
0,ATL,M,1912500.0,2297000.00,Miguel Almiron
1,ATL,D,65625.0,65625.00,Mikey Ambrose
2,ATL,M,150000.0,150000.00,Yamil Asad
3,ATL,D,99225.0,106573.89,Mark Bloom
4,ATL,F,65000.0,77400.00,Andrew Carleton


Merge Databases

In [273]:
mls_df = pd.merge(mmls_data_df, mls_salaries_df, how='inner', on=['Player'])
mls_df.head()

,Rk,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,...,Ast,G+A,G-PK,G+A-PK,Matches,Player,club,position,base_salary,guaranteed_compensation
0,1,us USA,DF,Sporting KC,25,1991,12,10,902,1,...,0.1,0.2,0.1,0.2,Matches,Kellyn Acosta,DAL,M,260000.0,280000.00
1,2,gh GHA,MF,Columbus,25,1991,20,14,1279,0,...,0,0,0,0,Matches,Luciano Acosta,DC,M-F,500000.0,602000.00
2,3,gh GHA,DF,Columbus,22,1994,7,6,529,1,...,0,0.17,0.17,0.17,Matches,Tyler Adams,NYRB,M,75000.0,91041.67
3,4,gh GHA,MFFW,Chicago,26,1990,30,24,2194,14,...,0.25,0.82,0.49,0.74,Matches,Seyi Adekoya,SEA,F,53004.0,53004.00
4,5,us USA,DF,Real Salt Lake,19,1997,17,16,1408,0,...,0.06,0.06,0,0.06,Matches,Fanendo Adi,POR,F,1190004.0,1736254.00


In [274]:
mls.dropna(how='any', inplace=True)
mls

,Rk,Pos,Squad,Age,Born,MP,Starts,Min,Gls,PK,...,Gls,G+A,G-PK,G+A-PK,Matches,Player,club,position,base_salary,guaranteed_compensation
0,1,DF,Sporting KC,25,1991,12,10,902,1,0,...,0.1,0.2,0.1,0.2,Matches,Kellyn Acosta,DAL,M,260000.0,280000.00
1,2,MF,Columbus,25,1991,20,14,1279,0,0,...,0,0,0,0,Matches,Luciano Acosta,DC,M-F,500000.0,602000.00
2,3,DF,Columbus,22,1994,7,6,529,1,0,...,0.17,0.17,0.17,0.17,Matches,Tyler Adams,NYRB,M,75000.0,91041.67
3,4,MFFW,Chicago,26,1990,30,24,2194,14,2,...,0.57,0.82,0.49,0.74,Matches,Seyi Adekoya,SEA,F,53004.0,53004.00
4,5,DF,Real Salt Lake,19,1997,17,16,1408,0,0,...,0,0.06,0,0.06,Matches,Fanendo Adi,POR,F,1190004.0,1736254.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,579,FW,NY Red Bulls,31,1985,32,29,2552,17,0,...,0.6,0.63,0.6,0.63,Matches,Eriq Zavaleta,TOR,D,102850.0,133450.00
417,580,DF,Philadelphia,22,1994,6,4,371,0,0,...,0,0,0,0,Matches,Ben Zemanski,POR,M,105000.0,109000.00
418,581,MF,Orlando City,26,1990,10,10,886,1,0,...,0.1,0.51,0.1,0.51,Matches,Walker Zimmerman,DAL,D,205000.0,205000.00
419,582,MFFW,San Jose,19,1997,13,10,969,0,0,...,0,0.09,0,0.09,Matches,Sal Zizzo,NYRB,M,110000.0,110000.00


Next Steps

Next Steps 2